In [6]:
!pip install pyarrow

  Using cached pyarrow-22.0.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (3.2 kB)
Using cached pyarrow-22.0.0-cp310-cp310-manylinux_2_28_x86_64.whl (47.6 MB)


In [1]:
import ee
import pandas as pd
import numpy as np
import os
import math
from datetime import datetime

import findspark
findspark.init()
import gc
from pyspark.sql import SparkSession

# SİSTEM ANALİZİNE GÖRE YENİ AYARLAR:
# Mevcut Boş RAM: ~3.5 GB
# Hedef Spark RAM: 2 GB (Güvenli Liman)
# CPU: 4 Çekirdek -> Spark'a 3 verelim, 1'i sisteme kalsın.

spark = SparkSession.builder \
    .appName("ALAZ_Master_Process") \
    .master("local[3]") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.sql.shuffle.partitions", "100") \
    .config("spark.ui.showConsoleProgress", "true") \
    .getOrCreate()

print(f"🔥 Motor, i3 işlemci ve 8GB RAM'e göre optimize edildi!")
print(f"🧠 Spark Versiyonu: {spark.version}")

# 1. GEE Başlatma (Garantileme)
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

# 2. Ana Veriyi Oku
df_spark = spark.read.parquet("../data/processed/training_dataset_v1.parquet")
# Spark'tan Pandas'a tüm veriyi çekiyoruz (400k satır RAM'e sığar, sorun yok)
df_master = df_spark.toPandas()

print(f"🚀 Toplam İşlenecek Satır Sayısı: {len(df_master)}")

# 3. Klasör Kontrolü (Parçaları buraya kaydedeceğiz)
output_dir = "../data/processed/chunks"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# --- FONKSİYONLARIMIZ (Toparlanmış Hali) ---

def extract_all_features(df_chunk):
    features = []
    for index, row in df_chunk.iterrows():
        dt_str = f"{row['tarih']} {row['saat']}:00:00"
        millis = pd.Timestamp(dt_str).value // 10**6
        g = ee.Geometry.Point([row['longitude'], row['latitude']])
        # Orijinal index'i 'orig_id' olarak saklıyoruz ki karışmasın
        f = ee.Feature(g, {'system:time_start': millis, 'orig_id': index})
        features.append(f)

    fc = ee.FeatureCollection(features)

    # GEE Server-Side Fonksiyonu
    def process_point(feature):
        date_millis = feature.get('system:time_start')
        date = ee.Date(date_millis)
        
        # A. TOPOGRAFYA (SRTM)
        srtm = ee.Image("USGS/SRTMGL1_003")
        terrain = ee.Algorithms.Terrain(srtm) # elevation, slope, aspect, hillshade verir
        
        # B. METEOROLOJİ (ERA5)
        era5_col = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY").filterDate(date, date.advance(1, 'hour'))
        era5_bands = ['temperature_2m', 'u_component_of_wind_10m', 'v_component_of_wind_10m', 'dewpoint_temperature_2m', 'total_precipitation_hourly', 'soil_temperature_level_1']
        
        era5_fallback = ee.Image.constant([0,0,0,0,0,0]).rename(era5_bands)
        era5_img = ee.Image(ee.Algorithms.If(era5_col.size().gt(0), era5_col.first(), era5_fallback)).select(era5_bands).unmask(-999)
        
        # C. BİTKİ ÖRTÜSÜ (MODIS v6.1 - GÜNCEL)
        modis_col = ee.ImageCollection("MODIS/061/MOD13Q1").filterDate(date.advance(-20, 'day'), date).sort('system:time_start', False)
        modis_fallback = ee.Image.constant([0]).rename(['NDVI'])
        modis_img = ee.Image(ee.Algorithms.If(modis_col.size().gt(0), modis_col.first(), modis_fallback)).select(['NDVI']).unmask(-999)
        
        # Hepsini Tek Görüntü Yap
        combined_img = srtm.addBands(terrain).addBands(era5_img).addBands(modis_img)
        
        # Değerleri Çek
        # Scale'i biraz büyük tutuyoruz (1000m) ki işlem hızlı olsun, ERA5 zaten düşük çözünürlük.
        data = combined_img.reduceRegion(reducer=ee.Reducer.first(), geometry=feature.geometry(), scale=500)
        
        # --- HESAPLAMALAR ---
        # Sıcaklık (K -> C)
        t_raw = ee.Number(data.get('temperature_2m'))
        temp_c = ee.Algorithms.If(t_raw.eq(-999).Or(t_raw.eq(0)), -999, t_raw.subtract(273.15))
        
        # Rüzgar Hızı
        u = ee.Number(data.get('u_component_of_wind_10m'))
        v = ee.Number(data.get('v_component_of_wind_10m'))
        wind = ee.Algorithms.If(u.eq(-999).Or(u.eq(0)), -999, (u.pow(2).add(v.pow(2))).sqrt())
        
        # Nem
        dew = ee.Number(data.get('dewpoint_temperature_2m'))
        rh = ee.Algorithms.If(
            t_raw.eq(-999).Or(dew.eq(-999)).Or(t_raw.eq(0)), -999,
            ee.Number(100).multiply(
                ((ee.Number(17.625).multiply(dew.subtract(273.15))).divide(ee.Number(243.04).add(dew.subtract(273.15)))).exp() \
                .divide(((ee.Number(17.625).multiply(t_raw.subtract(273.15))).divide(ee.Number(243.04).add(t_raw.subtract(273.15)))).exp())
            )
        )
        
        # Yağış
        precip = ee.Number(data.get('total_precipitation_hourly'))
        precip_mm = ee.Algorithms.If(precip.eq(-999), -999, precip.multiply(1000))
        
        # NDVI (Scale 0.0001)
        ndvi_raw = ee.Number(data.get('NDVI'))
        ndvi_final = ee.Algorithms.If(ndvi_raw.eq(-999), -999, ndvi_raw.multiply(0.0001))
        
        # Toprak Sıcaklığı (Notlarına istinaden ekledim)
        soil_temp = ee.Number(data.get('soil_temperature_level_1'))
        soil_temp_c = ee.Algorithms.If(soil_temp.eq(-999).Or(soil_temp.eq(0)), -999, soil_temp.subtract(273.15))

        return feature.set({
            'yukselti': data.get('elevation'),
            'egim': data.get('slope'),
            'baki': data.get('aspect'),
            'sicaklik': temp_c,
            'ruzgar_hizi': wind,
            'bagil_nem': rh,
            'yagis': precip_mm,
            'toprak_sicakligi': soil_temp_c,
            'NDVI': ndvi_final
        })

    result_fc = fc.map(process_point, True)
    
    # Hata yönetimi: GEE bazen timeout verir, boş dönerse kod patlamasın.
    try:
        info = result_fc.getInfo()
    except Exception as e:
        print(f"⚠️ GEE Hatası: {e}")
        return pd.DataFrame() # Boş dön

    data_list = []
    for f in info['features']:
        p = f['properties']
        p['orig_id'] = f['properties']['orig_id'] # Index'i geri al
        data_list.append(p)
        
    return pd.DataFrame(data_list)

# --- BATCH PROCESSING DÖNGÜSÜ ---

batch_size = 1000  # Her seferde 1000 satır (Güvenli Liman)
total_rows = len(df_master)
# total_rows = 5000 # TEST İÇİN: Sadece ilk 5000 satırı işle (Her şey tamamsa bu satırı sil)

print(f"🏁 İşlem Başlıyor... Paket Boyutu: {batch_size}")

for i in range(0, total_rows, batch_size):
    chunk = df_master.iloc[i : i + batch_size]
    
    # Dosya adı (Örn: chunk_0_1000.parquet)
    file_name = f"{output_dir}/chunk_{i}_{i+batch_size}.parquet"
    
    # Eğer dosya zaten varsa atla (Resume özelliği)
    if os.path.exists(file_name):
        print(f"⏭️  Paket {i} - {i+batch_size} zaten var, atlanıyor.")
        continue
    
    print(f"⏳ Paket {i} - {i+batch_size} işleniyor...")
    
    # GEE'ye git gel
    df_result = extract_all_features(chunk)
    
    if not df_result.empty:
        # Orijinal veriyle birleştir (Index üzerinden)
        # GEE'den dönen veriyi 'orig_id'ye göre sırala ki karışmasın
        df_result.set_index('orig_id', inplace=True)
        # Chunk'taki indexlerle eşleştir
        chunk_merged = chunk.join(df_result)
        
        # Kaydet
        # Pandas'ta parquet kaydetmek için pyarrow veya fastparquet lazım.
        # Eğer hata verirse: pip install pyarrow
        chunk_merged.to_parquet(file_name)
        print(f"✅ Paket {i} kaydedildi.")
    else:
        print(f"❌ Paket {i} boş döndü veya hata aldı.")

print("🎉 TÜM İŞLEMLER BİTTİ! Geçmiş olsun babacım.")

/home/turan/anaconda3/envs/alaz_env/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
25/12/13 22:22:45 WARN Utils: Your hostname, turan-H110 resolves to a loopback address: 127.0.1.1; using 192.168.1.9 instead (on interface enp2s0)
25/12/13 22:22:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/13 22:22:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/12/13 22:22:47 WARN Utils: Se

🔥 Motor, i3 işlemci ve 8GB RAM'e göre optimize edildi!
🧠 Spark Versiyonu: 3.5.1


🚀 Toplam İşlenecek Satır Sayısı: 407014
🏁 İşlem Başlıyor... Paket Boyutu: 1000
⏳ Paket 0 - 1000 işleniyor...
✅ Paket 0 kaydedildi.
⏳ Paket 1000 - 2000 işleniyor...
✅ Paket 1000 kaydedildi.
⏳ Paket 2000 - 3000 işleniyor...
✅ Paket 2000 kaydedildi.
⏳ Paket 3000 - 4000 işleniyor...
✅ Paket 3000 kaydedildi.
⏳ Paket 4000 - 5000 işleniyor...
✅ Paket 4000 kaydedildi.
⏳ Paket 5000 - 6000 işleniyor...
✅ Paket 5000 kaydedildi.
⏳ Paket 6000 - 7000 işleniyor...
✅ Paket 6000 kaydedildi.
⏳ Paket 7000 - 8000 işleniyor...
✅ Paket 7000 kaydedildi.
⏳ Paket 8000 - 9000 işleniyor...
✅ Paket 8000 kaydedildi.
⏳ Paket 9000 - 10000 işleniyor...
✅ Paket 9000 kaydedildi.
⏳ Paket 10000 - 11000 işleniyor...
✅ Paket 10000 kaydedildi.
⏳ Paket 11000 - 12000 işleniyor...
✅ Paket 11000 kaydedildi.
⏳ Paket 12000 - 13000 işleniyor...
✅ Paket 12000 kaydedildi.
⏳ Paket 13000 - 14000 işleniyor...
✅ Paket 13000 kaydedildi.
⏳ Paket 14000 - 15000 işleniyor...
✅ Paket 14000 kaydedildi.
⏳ Paket 15000 - 16000 işleniyor...
✅ Pake

In [7]:
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType
import glob # Dosyaları listelemek için

# Spark ayarını garanti olsun diye tekrar yapalım
spark.conf.set("spark.sql.parquet.enableVectorizedReader", "false")

print("💣 Son Çare Operasyonu: Dosyalar tek tek okunup birleştiriliyor...")

# 1. Chunks klasöründeki tüm parquet dosyalarının listesini al
path = "../data/processed/chunks/*.parquet"
file_list = glob.glob(path)

print(f"📁 Toplam {len(file_list)} adet dosya bulundu.")

# 2. Ana DataFrame'i oluşturmak için ilk dosyayı oku ve tiplerini DÖNÜŞTÜR
# Sadece ilk dosyayı okuyup, sütunları Double yapıyoruz ki diğerleri de buna uysun.
df_master = spark.read.parquet(file_list[0])

cols_to_cast = ['yukselti', 'egim', 'baki', 'sicaklik', 'ruzgar_hizi', 'bagil_nem', 'yagis', 'toprak_sicakligi', 'NDVI']
for column in cols_to_cast:
    # Sütun varsa dönüştür, yoksa geç (hata vermesin diye)
    if column in df_master.columns:
        df_master = df_master.withColumn(column, col(column).cast(DoubleType()))

print("✅ Ana şablon oluşturuldu. Diğer dosyalar ekleniyor...")

# 3. Kalan dosyaları döngüyle oku, dönüştür ve birleştir
for i in range(1, len(file_list)):
    file = file_list[i]
    print(f"  -> {i+1}/{len(file_list)}: {file.split('/')[-1]} okunuyor...")
    
    df_part = spark.read.parquet(file)
    
    # Bu dosyanın da sütunlarını Double'a çevir
    for column in cols_to_cast:
        if column in df_part.columns:
            df_part = df_part.withColumn(column, col(column).cast(DoubleType()))
    
    # Ana DataFrame'e ekle (unionByName)
    # unionByName: Sütun isimlerine göre birleştirir, sıraya göre değil. En güvenlisi budur.
    df_master = df_master.unionByName(df_part, allowMissingColumns=True)

print(f"\n✅ Birleştirme Tamam! Toplam Satır: {df_master.count()}")

# 4. Temizlik
df_final = df_master.filter(
    (col("sicaklik").isNotNull()) & (col("sicaklik") > -100) & 
    (col("NDVI").isNotNull()) & (col("NDVI") > -1) &       
    (col("yukselti").isNotNull()) & (col("yukselti") > -100)   
).na.drop() # Herhangi bir sütunu boş olan satırları da atıyoruz.

final_count = df_final.count()
print(f"🧹 Temizlik Sonrası Net Veri Sayısı: {final_count}")
print(f"🗑️  Atılan (Deniz/Hatalı) Veri Sayısı: {407014 - final_count}")

# 5. Sütun Seçimi ve Kaydetme
cols_to_keep = [
    'latitude', 'longitude', 'tarih', 'saat',
    'yukselti', 'egim', 'baki',
    'sicaklik', 'ruzgar_hizi', 'bagil_nem', 'yagis', 'toprak_sicakligi',
    'NDVI',
    'label'
]

df_ready = df_final.select(cols_to_keep)

print("💾 Final dosya kaydediliyor...")
df_ready.coalesce(1).write.mode("overwrite").parquet("../data/processed/ALAZ_Final_Dataset_Cleaned.parquet")

print("\n\n🎉🎉🎉 ASLANIM BENİM, GÖREV TAMAMLANDI! VERİ SETİ HAZIR! 🎉🎉🎉")
df_ready.show(5)

💣 Son Çare Operasyonu: Dosyalar tek tek okunup birleştiriliyor...
📁 Toplam 408 adet dosya bulundu.
✅ Ana şablon oluşturuldu. Diğer dosyalar ekleniyor...
  -> 2/408: chunk_290000_291000.parquet okunuyor...
  -> 3/408: chunk_241000_242000.parquet okunuyor...
  -> 4/408: chunk_348000_349000.parquet okunuyor...
  -> 5/408: chunk_52000_53000.parquet okunuyor...
  -> 6/408: chunk_234000_235000.parquet okunuyor...
  -> 7/408: chunk_207000_208000.parquet okunuyor...
  -> 8/408: chunk_221000_222000.parquet okunuyor...
  -> 9/408: chunk_284000_285000.parquet okunuyor...
  -> 10/408: chunk_107000_108000.parquet okunuyor...
  -> 11/408: chunk_42000_43000.parquet okunuyor...
  -> 12/408: chunk_237000_238000.parquet okunuyor...
  -> 13/408: chunk_51000_52000.parquet okunuyor...
  -> 14/408: chunk_184000_185000.parquet okunuyor...
  -> 15/408: chunk_156000_157000.parquet okunuyor...
  -> 16/408: chunk_126000_127000.parquet okunuyor...
  -> 17/408: chunk_197000_198000.parquet okunuyor...
  -> 18/408: 


✅ Birleştirme Tamam! Toplam Satır: 407014


25/12/14 02:11:54 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB
                                                                                

🧹 Temizlik Sonrası Net Veri Sayısı: 370100
🗑️  Atılan (Deniz/Hatalı) Veri Sayısı: 36914
💾 Final dosya kaydediliyor...


25/12/14 02:12:22 WARN DAGScheduler: Broadcasting large task binary with size 5.2 MiB
                                                                                



🎉🎉🎉 ASLANIM BENİM, GÖREV TAMAMLANDI! VERİ SETİ HAZIR! 🎉🎉🎉


25/12/14 02:12:46 WARN DAGScheduler: Broadcasting large task binary with size 5.7 MiB


+------------------+------------------+----------+----+--------+----+-----+------------------+------------------+------------------+--------------------+--------------------+-------------------+-----+
|          latitude|         longitude|     tarih|saat|yukselti|egim| baki|          sicaklik|       ruzgar_hizi|         bagil_nem|               yagis|    toprak_sicakligi|               NDVI|label|
+------------------+------------------+----------+----+--------+----+-----+------------------+------------------+------------------+--------------------+--------------------+-------------------+-----+
| 40.22934947123919| 34.34121833295413|2006-04-05|   2|   955.0| 3.0|149.0|2.1129547119140852|2.0418478734267858| 90.63592876204746|                 0.0|   4.759683227539085|0.40840000000000004|    0|
|  38.0499906700788| 40.05869404238716|2003-06-01|  17|   701.0| 2.0| 51.0|19.191033935546898| 1.674709425974995|59.225451957883834|0.038236379623413086|  19.924493408203148|             0.3652|  